In [19]:
import os
import numpy as np

from nasbench import api
from random_search import run_random_search, random_spec

from neural_predictor import classifier, regressor
from input_preprocessing import preprocess_nasbench
from search_spaces import load_nasbench_101

from scipy.stats import kendalltau as tau
from sklearn.metrics import r2_score, mean_squared_error

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [3]:
nasbench = load_nasbench_101()

Loading dataset from file... This may take a few minutes...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Loaded dataset in 57 seconds


In [4]:
N = 172
def get_N_samples(N):
    models = []

    for _ in range(N):
        while True:
            model = random_spec(nasbench)
            if model not in models:
                models.append(nasbench.query(model))
                break
                
    return preprocess_nasbench(models)

In [5]:
train_clf_data = get_N_samples(N)
clf = classifier([train_clf_data['X'], train_clf_data['norm_A'], train_clf_data['norm_AT']], train_clf_data['labels'])

In [6]:
np.save('outputs/train_clf_data_val_acc.npy', train_clf_data['val_acc'])

In [7]:
test_clf_data = get_N_samples(200000)

In [8]:
clf.evaluate([test_clf_data['X'], test_clf_data['norm_A'], test_clf_data['norm_AT']], test_clf_data['labels'])

6250/6250 [==============================] - 21s 3ms/step - loss: 0.2715 - accuracy: 0.9624


[0.2715485692024231, 0.9624300003051758]

In [9]:
y_pred = clf.predict([test_clf_data['X'], test_clf_data['norm_A'], test_clf_data['norm_AT']])

In [10]:
y_pred = np.where(y_pred > 0.5, 1, 0).ravel()

In [11]:
accurate_idx = np.where(y_pred == 1)[0]
inaccurate_idx = np.where(y_pred == 0)[0]

In [12]:
acc_set = test_clf_data['val_acc'][accurate_idx]
inacc_set = test_clf_data['val_acc'][inaccurate_idx]

In [13]:
np.save('outputs/clf_acc_set.npy', acc_set)
np.save('outputs/clf_inacc_set.npy', inacc_set)

In [14]:
models = get_N_samples(119+100000)

In [20]:
X_train, A_train, AT_train, y_train = models['X'][:79], models['norm_A'][:79], models['norm_AT'][:79], models['val_acc'][:79]

f_results = clf.predict([X_train, A_train, AT_train])
f_idx = np.where(f_results > 0.5)[0]

reg = regressor([X_train[f_idx], A_train[f_idx], AT_train[f_idx]], y_train[f_idx])

In [21]:
y_pred = reg.predict([X_train[f_idx], A_train[f_idx], AT_train[f_idx]]).ravel()
y_true = y_train[f_idx]

print(mean_squared_error(y_true, y_pred), tau(y_true, y_pred).correlation, r2_score(y_true, y_pred), y_pred.shape)

0.2587941525114619 0.6571079518894848 0.6900641242487546 (38,)


In [22]:
np.save('outputs/nasbench_two_y_true_train.npy', y_true)
np.save('outputs/nasbench_two_y_pred_train.npy', y_pred)
np.save('outputs/nasbench_two_mse_train.npy', mean_squared_error(y_true, y_pred))
np.save('outputs/nasbench_two_tau_train.npy', tau(y_true, y_pred).correlation)
np.save('outputs/nasbench_two_r2_train.npy', r2_score(y_true, y_pred))

In [23]:
X_val, A_val, AT_val, y_val = models['X'][79:119], models['norm_A'][79:119], models['norm_AT'][79:119], models['val_acc'][79:119]

f_results = clf.predict([X_val, A_val, AT_val])
f_idx = np.where(f_results > 0.5)[0]

y_pred = reg.predict([X_val[f_idx], A_val[f_idx], AT_val[f_idx]]).ravel()
y_true = y_val[f_idx]
print(mean_squared_error(y_true, y_pred), tau(y_true, y_pred).correlation, r2_score(y_true, y_pred), y_pred.shape)

0.7284672407902804 0.5032549801091942 0.1435993866266998 (22,)


In [24]:
np.save('outputs/nasbench_two_y_true_val.npy', y_true)
np.save('outputs/nasbench_two_y_pred_val.npy', y_pred)
np.save('outputs/nasbench_two_mse_val.npy', mean_squared_error(y_true, y_pred))
np.save('outputs/nasbench_two_tau_val.npy', tau(y_true, y_pred).correlation)
np.save('outputs/nasbench_two_r2_val.npy', r2_score(y_true, y_pred))

In [25]:
X_test, A_test, AT_test, y_test = models['X'][119:], models['norm_A'][119:], models['norm_AT'][119:], models['val_acc'][119:]

f_results = clf.predict([X_test, A_test, AT_test])
f_idx = np.where(f_results > 0.5)[0]

y_pred = reg.predict([X_test[f_idx], A_test[f_idx], AT_test[f_idx]]).ravel()
y_true = y_test[f_idx]

In [26]:
min_idx = np.argmin(y_pred.ravel())
max_idx = np.argmax(y_pred.ravel())

In [27]:
y_idx = np.where(y_pred > y_pred[min_idx])[0]

In [28]:
y_pred_10 = np.hstack([y_pred[min_idx], y_pred[list(range(0, len(y_idx), len(y_idx) // 8))[-8:]], y_pred[max_idx]]).ravel()
y_true_10 = np.hstack([y_true[min_idx], y_true[list(range(0, len(y_idx), len(y_idx) // 8))[-8:]], y_true[max_idx]]).ravel()
print(mean_squared_error(y_true_10, y_pred_10), tau(y_true_10, y_pred_10).correlation, r2_score(y_true_10, y_pred_10))

1.6367411297464116 0.7333333333333333 0.36024424501046204


In [29]:
np.save('outputs/nasbench_two_y_true_test.npy', y_true)
np.save('outputs/nasbench_two_y_pred_test.npy', y_pred)
np.save('outputs/nasbench_two_y_true_test_10.npy', y_true_10)
np.save('outputs/nasbench_two_y_pred_test_10.npy', y_pred_10)
np.save('outputs/nasbench_two_mse_test.npy', mean_squared_error(y_true, y_pred))
np.save('outputs/nasbench_two_tau_test.npy', tau(y_true, y_pred).correlation)
np.save('outputs/nasbench_two_r2_test.npy', r2_score(y_true, y_pred))